In [1]:
import requests
from lxml import etree
import pymysql

def get_html(url,time=10):
    try:
        head={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36"}
        
        resp=requests.get(url,headers=head,timeout=time)#发送请求
        
        resp.encoding=resp.apparent_encoding#设置返回内容的字符集编码
        #resp.encoding="utf-8"
        
        resp.raise_for_status()#返回的状态码不等于200抛出异常
        
        return resp.text#返回网页的文本内容
    except Exception as error:
        print(error)
info=[]
def parser(html):
    tree=etree.HTML(html)
    for row in tree.xpath("//div[@id='main']/div[3]/div[1]/a"):
        
        course_name=row.xpath("./p[1]/text()")[0].strip()#课程的名称
        
        learn_num=row.xpath("./p[2]/text()")[0].strip().split("·")[1].split("人")[0]#学习人数
        
        course_url='http:'+row.xpath("./@href")[0]#课程链接
        
        img_url='http:'+row.xpath("./div/@style")[0].split("('")[1].split("')")[0]#课程的图片url
        
        info.append((course_name,learn_num,course_url,img_url))
        
def save_mysql(sql,val,**dbinfo):  # 将爬取的内容存入mysql数据库
    try:
        con = pymysql.connect(**dbinfo)  # 连接mysql数据库
        cur = con.cursor()  # 创建游标对象
        cur.executemany(sql,val)
        con.commit()
    except Exception as e:  # 若报错，则获得错误类型
        con.rollback()
        print(e)
    finally:
        cur.close()  # 关闭游标
        con.close()  # 关闭链接        
        
if __name__=="__main__":
    for i in range(1,8):#爬取前7页的数据(完成拓展任务)
        if i==1:
            url='http://www.imooc.com/course/list'#爬取第一页的数据
        else:
            url='http://www.imooc.com/course/list?page={}'.format(str(i))#爬取第一页之后的所有页的数据
        html=get_html(url,time=10)
        if html==None:
            print("已爬取全部数据!\n")
            break
        else:
            parser(html)
    parms={
        "host":"localhost",
        "user":"root",
        "password": "root",
        "database":"study",
        "port":3306
    }
    sql='insert into mooc(c_name,l_num,c_url,i_url) values(%s,%s,%s,%s)'
    save_mysql(sql,info,**parms)
    print("已存储全部数据！")
    

NameError: name 'info' is not defined

In [ ]:
# import pymysql
# def save_mysql(sql,val,**dbinfo):  # 将爬取的内容存入mysql数据库
#     try:
#         con = pymysql.connect(**dbinfo)  # 1.创建数据库连接对象
        
#         cur = con.cursor()  # 2.获取游标对象
        
#         cur.executemany(sql,val) # 3.执行SQL语句（执行多条SQL）
#         #cur.execute(sql,val) #（执行一条SQL）
        
#         con.commit() # 4.事务提交
        
#     except Exception as e:  # 若报错，则返回错误类型
        
#         con.rollback() # 事务回滚
#         print(e)
        
#     finally:
#         cur.close()  # 5.关闭游标
#         con.close()  # 5.关闭连接        
        
# if __name__=="__main__":
#     #url='http://www.imooc.com/course/list'
#     #html=get_html(url,time=10)
#     #parser(html)
#     parms={
#         "host":"localhost",
#         "user":"root",
#         "password": "root",
#         "database":"study",
#         "port":3306
#     }
#     sql='insert into mooc(c_name,l_num,c_url,i_url) values(%s,%s,%s,%s)'
#     save_mysql(sql,info,**parms)
    